In [2]:
# Install tmdbsimple (only need to run once)
# !pip install tmdbsimple
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, time
from tqdm.notebook import tqdm_notebook

In [3]:
# Load TMDB API Key & Add to tmdbsimple
with open('.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Importing tmdbsimple and setting the API_KEY
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [5]:
# Designate a Folder

# Create the folder for saving files (if it doesn't exist)
FOLDER = "MovieData/"
os.makedirs(FOLDER, exist_ok=True)

# Show the list of files included in the folder
sorted(os.listdir(FOLDER))
['.DS_Store', '.ipynb_checkpoints', 'title-basics.csv']

# Define Your Functions
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info


def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

### Load in the cleaned Title Basics (from part 1)

In [6]:
basics = pd.read_csv('Data/basics.csv')
basics

# Set the year to filter for
YEARS_TO_GET = [2001,2002]

# Create an empty list for saving errors
errors = []

In [7]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    # Define the JSON file to store results for the year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if the JSON file exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    # If it exists, print a message
    else:
        print(f'The file {JSON_FILE} already exists.')

    # Filtering for movies from selected startYear
    df = basics.loc[basics['startYear']==YEAR].copy()

    # saving movie ids to list
    movie_ids = df['tconst']

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # Loop through movie_ids_to_get with a tqdm progress bar
    for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  # This uses your pre-made function
            
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
    
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            errors.append([movie_id, e])
        
    # Save the final results to a csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
    final_year_df.to_csv(csv_fname, compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

The file MovieData/tmdb_api_results_2001.json already exists.


Movies from 2001:   0%|          | 0/224 [00:00<?, ?it/s]

The file MovieData/tmdb_api_results_2002.json already exists.


Movies from 2002:   0%|          | 0/280 [00:00<?, ?it/s]

In [8]:
print(f"- Total errors: {len(errors)}")

- Total errors: 504
